In [1]:
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from joblib import dump, load
import geopandas as gpd
import xgboost as xgb
import numpy as np
import pandas as pd

In [2]:
pipe_rf = load('./models/cv_rf.joblib') 
pipe_lr = load('./models/cv_lr.joblib') 
pipe_xgb = load('./models/cv_xgb.joblib') 

/home/laia/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/laia/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/laia/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid re

In [3]:
train = gpd.read_file( "./data/train_data_final.geojson")
test =  gpd.read_file("./data/train_data_final.geojson")

In [4]:
X = train.loc[:,'NDVI_2000':'NDVI_2019']
y = train['label.x']
X_test = test.loc[:,'NDVI_2000':'NDVI_2019']

In [ ]:
best_model_rf = pipe_rf.best_estimator_
best_model_lr = pipe_lr.best_estimator_
best_model_xgb = pipe_xgb.best_estimator_

In [ ]:
print("Classification report de RF")
print(classification_report(test.label_0,best_model_rf.predict(X_test)))
print("\n")
print("Classification report de reg logística")
print(classification_report(test.label_0,best_model_lr.predict(X_test)))
print("\n")
print("Classification report de XGBoost")
print(classification_report(test.label_0,best_model_xgb.predict(X_test)))
print("\n")

Classification report de RF
              precision    recall  f1-score   support

           0       0.67      0.55      0.61       877
           1       0.97      0.98      0.97      4964
           2       0.69      0.77      0.72       859

    accuracy                           0.90      6700
   macro avg       0.78      0.77      0.77      6700
weighted avg       0.89      0.90      0.89      6700





In [ ]:
test['predictions_rf'] = best_model_rf.predict(X_test)
test['predictions_lr'] = best_model_lr.predict(X_test)
test['predictions_xgb'] = best_model_xgb.predict(X_test)

In [ ]:
#test.to_file("data/test_predictions.geojson", driver='GeoJSON')
#Para GEE en .shp
test.to_file("data/preds/test_predictions.shp")


/tmp/ipykernel_15275/3032649008.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  test.to_file("data/preds/test_predictions_rf_new.shp")


In [21]:
def exporto_clasif(modelo, nombre):
    report_data = classification_report(test.label_0,modelo.predict(X_test), output_dict=True)
    dataframe = pd.DataFrame.from_dict(report_data).reset_index()
    dataframe = dataframe.rename(columns={"index": "metric"})
    
    dataframe.to_csv('data/preds/classification_report_'+nombre+'.csv', index = False)

In [22]:
exporto_clasif(best_model_rf, 'random_forest')
exporto_clasif(best_model_lr, 'reg_log')
exporto_clasif(best_model_xgb, 'xgb')